In [ ]:
import os
import pandas as pd
import geopandas as gpd

import rasterio

import pystac_client
import planetary_computer as pc

import lidar_sampling_functions as lsf

In [ ]:
# accesing Azure storage using pystac client
URL = "https://planetarycomputer.microsoft.com/api/stac/v1"
catalog = pystac_client.Client.open(URL)

#campus point 2018 naip scene
itemid = 'ca_m_3411934_sw_11_060_20180722_20190209'
search = catalog.search(
    collections=["naip"],
    ids = itemid
)
item = list(search.get_items())[0]
# sign and open item
href = pc.sign(item.assets["image"].href)
ds = rasterio.open(href)
ds.crs

In [ ]:
fp = '/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/campus_lagoon_points/campus_lagoon_points_2018.csv'
pts = lsf.geodataframe_from_csv(fp, ds.crs)

In [ ]:
pts.head(3)

In [ ]:
fp = os.path.join(os.getcwd(),'3mlidar_over_campuspoint_naip_scene.tif')
lidar_rast_r = rasterio.open(fp)

In [ ]:
# lsf.save_min_max_diff_rasters(lidar_rast_r, os.getcwd())
lsf.save_avg_rasters(lidar_rast_r, os.getcwd())

In [ ]:
pts_xy = lsf.pts_for_lidar_sampling(pts, lidar_rast_r.crs)

In [ ]:
lidar_samples = lsf.sample_raster(pts_xy, lidar_rast_r)

In [ ]:
fp = os.path.join(os.getcwd(),'lidar_maxs.tif')
maxs_rast_r = rasterio.open(fp)
max_samples = lsf.sample_raster(pts_xy, maxs_rast_r)

In [ ]:
fp = os.path.join(os.getcwd(),'lidar_mins.tif')
mins_rast_r = rasterio.open(fp)
min_samples = lsf.sample_raster(pts_xy, mins_rast_r)

In [ ]:
fp = os.path.join(os.getcwd(),'lidar_avgs.tif')
avg_rast_r = rasterio.open(fp)
avg_samples = lsf.sample_raster(pts_xy, avg_rast_r)

In [ ]:
pts['lidar'] = lidar_samples
pts['max_lidar_w']= max_samples
pts['min_lidar_w'] = min_samples
pts['min_max_diff'] = pts.max_lidar_w - pts.min_lidar_w
pts['avg_lidar_w'] = avg_samples


In [ ]:
pts.head(3)

## Some stats

In [ ]:
ice = pts.loc[pts.iceplant==1]
nonice = pts.loc[pts.iceplant==0]

In [ ]:
ice.lidar.value_counts()

In [ ]:
ice.min_max_diff.plot.hist(grid=True, bins=25, rwidth=0.9)

In [ ]:
nonice.min_max_diff.plot.hist(grid=True, bins=25, rwidth=0.9)

In [ ]:
ice.max_lidar_w.plot.hist(grid=True, bins=25, rwidth=0.9)

In [ ]:
nonice.max_lidar_w.plot.hist(grid=True, bins=25, rwidth=0.9)

In [ ]:
ice.avg_lidar_w.plot.hist(grid=True, bins=25, rwidth=0.9)

In [ ]:
nonice.avg_lidar_w.plot.hist(grid=True, bins=25, rwidth=0.9)

In [ ]:
nonice.avg_lidar_w.describe()